In [2]:
from collections import namedtuple
import argparse
import os
import sys
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset

In [8]:
def to_variable(tensor):
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)

In [7]:
args = namedtuple('args', ['batch_size', 'wordcount', 'epochs', 'cuda', 'embedding_dim', 'hidden_dim', 'clip', 'num_layers'])(120, 7614, 10, True, 100, 1000, 0.25, 1)



In [12]:
class LSTM(nn.Module):
    def __init__(self, args):
        super(LSTM, self).__init__()
        self.wordcount = args.wordcount
        self.embedding = nn.Embedding(num_embeddings=args.wordcount, embedding_dim=args.embedding_dim)
        self.embedding.weight.data.uniform_(-0.8, 0.8)
        self.lstm1 = nn.LSTM(input_size=args.embedding_dim, num_layers=args.num_layers, hidden_size=args.hidden_dim, batch_first=True)
        for name, param in self.lstm1.named_parameters():
            if 'bias' in name:
                init.constant(param, 0.0)
            elif 'weight' in name:
                init.uniform(param, -0.8, 0.8)
        self.projection = nn.Linear(in_features=args.embedding_dim, out_features=args.wordcount)
        self.projection.weight = self.embedding.weight

    def forward(self, input, lengths , stochastic=False):
        embeddings = self.embedding(input)
        packed = pack_padded_sequence(embeddings, lengths)
        h_t, c_t = self.lstm1(packed)
        output, _ = pad_packed_sequence(h_t)
        output = self.projection(output)
        return output
